In [ ]:
!pip -q install /kaggle/input/pytorchtabnet/pytorch_tabnet-4.1.0-py3-none-any.whl

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import random
from copy import deepcopy
from sklearn.base import clone, BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.callbacks import Callback

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.callbacks import Callback
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
import seaborn as sns
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

SEED = 18
n_splits = 5

# Hàm xử lý time-series

### Mô tả mã nguồn

- **`process_file(filename, dirname)`**:  
  Hàm này đọc từng file dữ liệu time-series trong định dạng Parquet, xóa cột không cần thiết (`step`), và tính toán các thống kê mô tả như trung bình, phương sai, tối đa, tối thiểu, v.v. Kết quả trả về gồm:  
  1. **Các giá trị thống kê** dưới dạng mảng.  
  2. **ID của file**, được tách từ tên file.

- **`load_time_series(dirname)`**:  
  Hàm này xử lý toàn bộ thư mục chứa dữ liệu time-series bằng cách:
  1. Lấy danh sách tất cả các file trong thư mục (`ids`).  
  2. Sử dụng `ThreadPoolExecutor` để xử lý đa luồng, gọi `process_file` trên từng file, tăng tốc độ xử lý.  
  3. Kết hợp các kết quả từ tất cả file thành một DataFrame. Mỗi hàng tương ứng với một file, các cột là các giá trị thống kê được gán nhãn như `stat_0`, `stat_1`,...  
  4. Thêm cột `id` để lưu trữ ID tương ứng với mỗi hàng.

### Ý nghĩa:
- **Tự động hóa xử lý dữ liệu time-series:**  
  Giảm kích thước dữ liệu time-series bằng cách chỉ giữ lại các đặc trưng quan trọng (thống kê), giúp tăng tốc độ xử lý và huấn luyện mô hình.

- **Tăng hiệu quả tính toán:**  
  Việc sử dụng `ThreadPoolExecutor` giúp xử lý đa luồng, tiết kiệm thời gian khi làm việc với nhiều file.

- **Đầu ra:**  
  DataFrame chứa các đặc trưng thống kê của toàn bộ dữ liệu time-series, sẵn sàng để tích hợp vào dữ liệu chính.


In [ ]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df


# AutoEncoder

### Mô tả

#### **`class AutoEncoder`**
- **Encoder**: Giảm chiều dữ liệu từ `input_dim` xuống `encoding_dim` qua các tầng `Linear` với hàm kích hoạt `ReLU`.
- **Decoder**: Tái tạo dữ liệu từ `encoding_dim` về `input_dim` qua các tầng `Linear`, dùng `Sigmoid` ở tầng cuối.
- **`forward`**: Trích xuất đặc trưng qua `encoder` và tái tạo dữ liệu qua `decoder`.

#### **`perform_autoencoder`**
- **Chuẩn hóa dữ liệu**: Chuẩn hóa bằng `StandardScaler`, chuyển đổi sang Tensor.
- **Huấn luyện**: Sử dụng `MSELoss` để tối ưu việc tái tạo dữ liệu qua AutoEncoder, cập nhật trọng số bằng Adam.
- **Trích xuất đặc trưng**: Lấy đầu ra từ `encoder` để tạo DataFrame các đặc trưng mã hóa (`Enc_1`, `Enc_2`,...).

### Ý nghĩa
- **Giảm chiều dữ liệu**: Tự động học đặc trưng quan trọng với ít chiều hơn.
- **Tăng hiệu quả tính toán**: Chuẩn bị dữ liệu cho các mô hình sau.
- **Đầu ra**: DataFrame chứa đặc trưng mã hóa, sẵn sàng sử dụng.


In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim*3),
            nn.ReLU(),
            nn.Linear(input_dim*3, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

# Feature engineering

In [ ]:
def feature_engineering(df):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    # Create engineered features
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

# Hàm Feature Engineering

### Mô tả

#### **Xử lý dữ liệu**
- **Xóa các cột mùa (`Season`)**: Loại bỏ thông tin mùa vì không cần thiết cho mô hình.

#### **Tạo đặc trưng mới**
- **Kết hợp các cột hiện có để tạo đặc trưng ý nghĩa hơn:**
  - `BMI_Age`: Kết hợp giữa chỉ số BMI và tuổi.
  - `Internet_Hours_Age`: Số giờ sử dụng internet nhân với tuổi.
  - `BMI_Internet_Hours`: Tích hợp BMI và thời gian sử dụng internet.
  - Tỷ lệ:
    - `BFP_BMI`: Tỷ lệ phần trăm mỡ cơ thể (BFP) so với BMI.
    - `FFMI_BFP`, `FMI_BFP`: Các chỉ số về cơ và mỡ.
    - `LST_TBW`: Tỷ lệ cơ xương nạc trên tổng lượng nước cơ thể.
  - Tích: 
    - `BFP_BMR`, `BFP_DEE`: Liên kết BFP với chuyển hóa cơ bản (BMR) và năng lượng tiêu hao (DEE).
  - Tỷ lệ khác:
    - `BMR_Weight`, `DEE_Weight`: Điều chỉnh BMR và DEE theo trọng lượng.
    - `SMM_Height`: Tỷ lệ khối cơ xương so với chiều cao.
    - `Muscle_to_Fat`: Tỷ lệ cơ trên mỡ.
    - `Hydration_Status`, `ICW_TBW`: Các chỉ số liên quan đến nước và cân bằng trong cơ thể.

### Ý nghĩa
- **Tăng thông tin hữu ích**: Giúp mô hình học được các mối quan hệ phức tạp hơn giữa các đặc trưng.
- **Cải thiện hiệu năng**: Cung cấp thêm ngữ cảnh và ý nghĩa từ dữ liệu ban đầu.
- **Đầu ra**: DataFrame với các đặc trưng mới được tạo và các cột không cần thiết đã loại bỏ.


In [ ]:
# Định nghĩa hàm tính Quadratic Weighted Kappa
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

# Hàm làm tròn dự đoán dựa trên các ngưỡng
def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

# Hàm đánh giá dự đoán với các ngưỡng
def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# Hàm `TrainModel`

### **Mô tả**
Hàm này thực hiện quá trình huấn luyện, đánh giá và tối ưu hóa mô hình.

---

### **Các bước chính**

1. **Chuẩn bị dữ liệu**:
   - Tách dữ liệu `train` thành `X` (đặc trưng) và `y` (nhãn `sii`).
   - Sử dụng **Stratified K-Fold** để chia dữ liệu thành `n_splits` tập huấn luyện và kiểm tra.

2. **Huấn luyện và kiểm tra**:
   - Với mỗi fold:
     - Huấn luyện mô hình trên tập huấn luyện (`train_idx`).
     - Dự đoán và tính toán độ đo **QWK (Quadratic Weighted Kappa)** trên tập kiểm tra (`test_idx`).
   - Lưu các kết quả:
     - `oof_non_rounded`: Dự đoán không làm tròn.
     - `oof_rounded`: Dự đoán sau khi làm tròn.
     - `test_preds`: Dự đoán trên dữ liệu kiểm tra.

3. **Tối ưu hóa ngưỡng**:
   - Sử dụng hàm `minimize` với phương pháp **Nelder-Mead** để tối ưu hóa các ngưỡng làm tròn, giúp cải thiện điểm **QWK**.

4. **Dự đoán cuối cùng**:
   - Kết hợp các dự đoán từ các fold và áp dụng ngưỡng tối ưu để tạo **submission** cuối cùng.

---

### **Đầu ra**
- **Kết quả trung bình trên các fold**:
  - `Mean Train QWK`: Điểm QWK trung bình trên tập huấn luyện.
  - `Mean Validation QWK`: Điểm QWK trung bình trên tập kiểm tra.
  - `Optimized QWK SCORE`: Điểm QWK sau khi áp dụng ngưỡng tối ưu.

- **Dự đoán cuối cùng**:
  - Dự đoán `sii` trên dữ liệu kiểm tra được lưu trong DataFrame `submission`.

---

### **Ý nghĩa**
- **Đảm bảo đánh giá công bằng**: Sử dụng Stratified K-Fold để đảm bảo phân phối nhãn đồng đều.
- **Tối ưu hóa kết quả**: Áp dụng kỹ thuật tối ưu ngưỡng giúp cải thiện điểm đánh giá.
- **Kết hợp dự đoán**: Lấy trung bình dự đoán từ các fold để tăng độ ổn định của kết quả.

---

In [ ]:
def TrainModel(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim*3),
            nn.ReLU(),
            nn.Linear(input_dim*3, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

# Bảng phân phối giá trị của Sii

In [ ]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
missing_count = train['sii'].isna().sum()
train['sii_filled'] = train['sii'].fillna('Missing')
percentages = train['sii_filled'].value_counts(normalize=True) * 100
ax = sns.countplot(data=train, x='sii_filled', order=percentages.index)
ax.set_title("Count and Percentage of 'sii' (Including Missing Values)")
ax.set_xlabel("Sii")
ax.set_ylabel("Count")
for p, percentage in zip(ax.patches, percentages):
    ax.annotate(f"{percentage:.1f}%", 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=10, color='black')

# Bảng thể hiện đặc trưng tương quan với PCIAT_Total

In [ ]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
PCIAT_cols = [val for val in train.columns[train.columns.str.contains('PCIAT')]]
train_cat_columns = train.select_dtypes(exclude='number').columns
PCIAT_cols.remove('PCIAT-PCIAT_Total') 
train = train.drop(columns = PCIAT_cols)
train = train.dropna(subset=['sii'])
numeric_cols = train.select_dtypes(include='number')
corr = numeric_cols.corr()['PCIAT-PCIAT_Total'].sort_values(ascending=False)
corr_df = corr.reset_index()
corr_df.columns = ['Feature', 'Correlation']
plt.figure(figsize=(10, 10))
sns.barplot(x='Correlation', y='Feature', data=corr_df, palette='coolwarm')
for index, row in corr_df.iterrows():
    plt.text(row['Correlation'] + 0.01, index, f"{row['Correlation']:.2f}", va='center')
plt.title('Các Đặc Trưng Tương Quan với PCIAT-PCIAT_Total', fontsize=16)
plt.xlabel('Hệ số tương quan', fontsize=14)
plt.ylabel('Đặc trưng', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

missing_train = train.isnull().mean() * 100
missing_train = missing_train.sort_values(ascending=False)

# Chỉ chọn các feature có tỉ lệ thiếu cao (ví dụ > 5%)
missing_train_filtered = missing_train[missing_train > 5]

plt.figure(figsize=(12, 20))
sns.barplot(x=missing_train_filtered.values, y=missing_train_filtered.index, palette='viridis')
plt.xlabel('Percentage of Missing Values (%)')
plt.ylabel('Features')
plt.title('Percentage of Missing Values per Feature in Training Set')
plt.show()

# Version 1

In [ ]:
#Model 1

# Tải dữ liệu
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# Tải và xử lý dữ liệu time series
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# Thêm các cột mã hóa thời gian vào dữ liệu chính
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

# Merge dữ liệu time series đã mã hóa với dữ liệu chính
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# Chọn cột đặc trưng
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

# Chọn các cột đặc trưng
train = train[featuresCols]
train = train.dropna(subset=['sii'])

# Định nghĩa các cột mùa
cat_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]

# Gán các giá trị cột mùa bị thiếu bằng Missing
def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

# Chuyển các dữ liệu sang số
season_mapping = {'Spring': 1, 'Summer': 2, 'Fall': 3, 'Winter': 4}
for col in cat_c:
    train[col] = train[col].map(season_mapping)
    test[col] = test[col].map(season_mapping)

imputer = SimpleImputer(strategy='median')

# Tạo các mô hình
ensemble = VotingRegressor(estimators=[
    ('lgb', Pipeline(steps=[('imputer', imputer), ('regressor', LGBMRegressor(random_state=SEED))])),
    ('xgb', Pipeline(steps=[('imputer', imputer), ('regressor', XGBRegressor(random_state=SEED))])),
    ('cat', Pipeline(steps=[('imputer', imputer), ('regressor', CatBoostRegressor(random_state=SEED))])),
])

# Train the ensemble model
Submission1 = TrainModel(ensemble, test)

# Version 2

In [ ]:
#Model 2

# Tải dữ liệu
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

# Tải và xử lý dữ liệu time series
train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

# Thêm các cột mã hóa thời gian vào dữ liệu chính
time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

# Merge dữ liệu time series đã mã hóa với dữ liệu chính
train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# Chọn cột đặc trưng
featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday']

featuresCols += time_series_cols

# Đảm bảo 'sii' nằm trong featuresCols trước khi chọn cột
featuresCols.append('sii')

# Chọn các cột đặc trưng
train = train[featuresCols]
train = train.dropna(subset=['sii'])

# Định nghĩa các cột phân loại
cat_c = [
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 
    'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 
    'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season'
]
# Cập nhật các cột phân loại
def update(df):
    global cat_c
    for c in cat_c: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df
        
train = update(train)
test = update(test)

# Mã hóa các cột phân loại thành số nguyên
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in cat_c:
    mapping = create_mapping(col, train)
    mappingTe = create_mapping(col, test)
    
    train[col] = train[col].replace(mapping).astype(int)
    test[col] = test[col].replace(mappingTe).astype(int)

# Định nghĩa các tham số cho các mô hình
lgb_Params = {
    'learning_rate': 0.046,
    'max_depth': 9,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 4,
    'verbose': -1,
    'lambda_l1': 10,
    'lambda_l2': 0.01,
}

XGB_Params = {
    'max_depth': 6,
    'n_estimators': 150,
    'learning_rate': 0.05,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED,
    'enable_categorical': True
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'cat_features': cat_c,
    'verbose': 0,
    'l2_leaf_reg': 10
}

# Tạo các mô hình
Light = LGBMRegressor(**lgb_Params)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Tạo mô hình Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
]) 

# Train the ensemble model
Submission2 = TrainModel(voting_model, test)

# Version 3

In [ ]:
# Tải dữ liệu
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)

train_ts_encoded = perform_autoencoder(df_train, encoding_dim=60, epochs=100, batch_size=32)
test_ts_encoded = perform_autoencoder(df_test, encoding_dim=60, epochs=100, batch_size=32)

# Thêm các cột mã hóa thời gian vào dữ liệu chính
time_series_cols = train_ts_encoded.columns.tolist()
train_ts_encoded["id"]=train_ts["id"]
test_ts_encoded['id']=test_ts["id"]

# Merge dữ liệu time series với dữ liệu chính
train = pd.merge(train, train_ts_encoded, how="left", on='id')
test = pd.merge(test, test_ts_encoded, how="left", on='id')

# Impute dữ liệu numeric bằng KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['int32', 'int64', 'float64']).columns
imputed_data = knn_imputer.fit_transform(train[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)
train_imputed['sii'] = train_imputed['sii'].round().astype(int)

for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]

train = train_imputed

# Áp dụng feature engineering
train = feature_engineering(train)
test = feature_engineering(test)

if 'id' in train.columns:
    train = train.drop('id', axis=1)
if 'id' in test.columns:
    test = test.drop('id', axis=1)

# Chọn cột đặc trưng
featuresCols = [
    'Basic_Demos-Age', 'Basic_Demos-Sex',
    'CGAS-CGAS_Score', 'Physical-BMI',
    'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
    'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
    'Fitness_Endurance-Max_Stage',
    'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
    'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
    'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
    'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
    'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
    'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
    'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
    'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
    'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
    'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total',
    'PAQ_C-PAQ_C_Total', 'SDS-SDS_Total_Raw',
    'SDS-SDS_Total_T',
    'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age', 'Internet_Hours_Age', 'BMI_Internet_Hours',
    'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW', 'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight',
    'SMM_Height', 'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW'
]



# Thêm các cột từ time series
featuresCols += time_series_cols
featuresCols.append('sii')

train = train[featuresCols]
train = train.dropna(subset='sii')

# Tương tự cho test
featuresCols = [c for c in featuresCols if c != 'sii']
test = test[featuresCols]

# Kiểm tra và xử lý các giá trị vô hạn trong dữ liệu huấn luyện
if np.any(np.isinf(train)):
    train = train.replace([np.inf, -np.inf], np.nan)

# Định nghĩa các tham số cho các mô hình
lgb_Params = {
    'learning_rate': 0.046,
    'max_depth': 9,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 4,
    'verbose': -1,
    'lambda_l1': 10,
    'lambda_l2': 0.01,
}

XGB_Params = {
    'max_depth': 6,
    'n_estimators': 150,
    'learning_rate': 0.05,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': SEED
}

CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10
}

# Tạo các mô hình
Light = LGBMRegressor(**lgb_Params)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Tạo mô hình Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
]) 

# Train the ensemble model
Submission3= TrainModel(voting_model, test)

# Kết quả

In [ ]:
sub1 = Submission1
sub1 = sub1.sort_values(by='id').reset_index(drop=True)

sub2 = Submission2
sub2 = sub2.sort_values(by='id').reset_index(drop=True)

sub3 = Submission3
sub3 = sub3.sort_values(by='id').reset_index(drop=True)

combined = pd.DataFrame({
    'id': sub1['id'],
    'sii_1': sub1['sii'],
    'sii_2': sub2['sii'],
    'sii_3': sub3['sii'],
})

def majority_vote(row):
    return row.mode()[0]

combined['final_sii'] = combined[['sii_1', 'sii_2', 'sii_3']].apply(majority_vote, axis=1)

final_submission = combined[['id', 'final_sii']].rename(columns={'final_sii': 'sii'})

# Save submission
final_submission.to_csv('submission.csv', index=False)
print("Save submission!")